# Image Class Project 2


## Introduction

## Choise of traking algorithm

In [ ]:
!pip install cvzone

In [18]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder

# Create the color Finder object
# True : debug mode
myColorFinder = ColorFinder(False)
hsvVals = {'hmin': 0, 'smin': 176, 'vmin': 51, 'hmax': 16, 'smax': 255, 'vmax': 255}

# Variables
posListX, posListY = [], []
xList = [item for item in range(0, 1300)]
prediction = False

cap = cv2.VideoCapture("Bouncing ball.mp4")

while True:
    # Grab the image
    success, img = cap.read()
    # Display
    # img = cv2.imread("ball.png")
    # cv2.imshow('img',img)
    # Process the image using the ColorFinder update method
    imgColor, mask = myColorFinder.update(img, hsvVals)
    
    imgContours, contours = cvzone.findContours(img,mask, minArea=200)

    imgColor = cv2.resize(imgColor, (0, 0), None, 0.4, 0.4)
    
    # cv2.imshow("Image with Color", imgColor)
    cv2.imshow("Video ball", imgContours)
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [1]:
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder

# Create the color Finder object
# True : debug mode
myColorFinder = ColorFinder(False)
hsvVals = {'hmin': 0, 'smin': 176, 'vmin': 51, 'hmax': 16, 'smax': 255, 'vmax': 255}

# Variables
posListX, posListY = [], []
xList = [item for item in range(0, 1300)]
prediction = False

cap = cv2.VideoCapture("Bouncing ball.mp4")

while True:
    # Grab the image
    success, img = cap.read()
    # Display
    # Process the image using the ColorFinder update method
    imgColor, mask = myColorFinder.update(img, hsvVals)
    
    imgContours, contours = cvzone.findContours(img,mask, minArea=200)

    if contours:
        cx, cy = contours[0]['center']
        print(cx, cy)
        cv2.circle(imgContours, (cx,cy), 5, (0, 255, 0), cv2.FILLED)
    imgColor = cv2.resize(imgContours, (0, 0), None, 0.4, 0.4)
    
    # cv2.imshow("Image with Color", imgColor)
    cv2.imshow("Video ball", imgContours)
    cv2.waitKey(50)
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

669 48
661 48
655 55
651 80
647 132
647 132
641 203
640 292
640 403
558 421
558 421
560 541
543 472
537 391
537 391
613 385
605 324
599 278
595 247
592 232
592 230
592 230
587 230
583 252
583 283
583 283
579 340
575 410
574 499
574 499
529 603
569 553
566 490
564 429
561 390
561 390
558 364
554 354
553 356
555 373
555 373
554 411
554 465
554 531
554 531
516 606
553 543
548 497
548 459
545 442
545 442
548 421
548 431
548 464
549 504
549 504
511 595
526 616
552 547
552 547
549 513
547 491
547 480
547 492
552 522
552 522
466 506
468 560
476 525
477 490
476 490
479 472
481 470
483 479
483 479
491 508
491 551
499 534
499 503
501 494
501 494
503 494
515 510
515 546
529 534
529 533
533 507
538 499
543 506
543 506
576 531
580 534
584 513
587 509
591 526
591 526
596 544
601 526
604 522
608 541
608 541
612 548
616 533
618 536
618 536
622 546
626 537
628 536
632 535
635 535
635 535
637 544
639 541
642 545
645 541
645 541
649 542
650 541
653 540
652 541
656 541
657 542
658 544
663 543
662 550
662 

In [21]:
import math
import cv2
import cvzone
from cvzone.ColorModule import ColorFinder
import numpy as np

# Initialize the Video
cap = cv2.VideoCapture('Bouncing ball.mp4')

# Create the color Finder object
myColorFinder = ColorFinder(False)
hsvVals = {'hmin': 0, 'smin': 176, 'vmin': 51, 'hmax': 16, 'smax': 255, 'vmax': 255}

# Variables
posListX, posListY = [], []
xList = [item for item in range(0, 1300)] # image shpae of x -> 1300x1080
prediction = False

while True:
    
    success, img = cap.read()
    # img = img[0:900, :]
    
    # Display
    img = cv2.resize(img, (0, 0), None, 0.7, 0.7)
    
    # Process the image using the ColorFinder update method
    imgColor, mask = myColorFinder.update(img, hsvVals)
    
    imgContours, contours = cvzone.findContours(img, mask, minArea=500)
    
    if contours:
        posListX.append(contours[0]['center'][0])
        posListY.append(contours[0]['center'][1])
        
    if posListX and posListY: 
        # polunomial regression y = Ax ^ 2 + Bx + C
        A, B, C = np.polyfit(posListX, posListY, 2)
        
        
        for i, (posX, posY) in enumerate(zip(posListX, posListY)):
            pos = (posX, posY)
            cv2.circle(imgContours, pos, 5, (0, 255, 0), cv2.FILLED)
            if i >= 1:
                cv2.line(imgContours, pos, (posListX[i - 1], posListY[i - 1]), (0, 255, 0), 2)
        
        for x in xList:
            y = int(A * x**2 + B * x + C)
            cv2.circle(imgContours, (x, y), 2, (255, 0, 255), cv2.FILLED)
            
        print(posListX)
        if len(posListX) < 10 :
            # Prediction
            a = A
            b = B
            c = C - 590

            x = int((-b - math.sqrt(b ** 2 - (4 * a * c))) / (2 * a))
            # print(x)
            prediction = 100 < x < 150

        if prediction:
            cvzone.putTextRect(imgContours, "Basket", (50, 100),
                               scale=3, thickness=3, colorR=(0, 200, 0), offset=20)
        else:
            cvzone.putTextRect(imgContours, "No Basket", (50, 100),
                               scale=3, thickness=3, colorR=(0, 0, 200), offset=20)
            
    cv2.imshow("Image with countours", imgContours)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(85) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

C:\Users\alexc\AppData\Local\Temp\ipykernel_33628\10617684.py:38: RankWarning: Polyfit may be poorly conditioned
  A, B, C = np.polyfit(posListX, posListY, 2)
C:\Users\alexc\AppData\Local\Temp\ipykernel_33628\10617684.py:38: RankWarning: Polyfit may be poorly conditioned
  A, B, C = np.polyfit(posListX, posListY, 2)


[468]
[468, 462]


ValueError: math domain error

## Step by step

In [1]:
 
import numpy as np 
import cv2 as cv 
  

In [2]:
# Read the input video 
cap = cv.VideoCapture('Bouncing ball.mp4') 
  
  

In [ ]:
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

print(f"Video Resolution: {frame_width}x{frame_height}")


In [4]:
# take first frame of the 
# video 
ret, frame = cap.read() 

In [5]:
# Manually select ROI
roi = cv.selectROI("Select Tracking Region", frame, fromCenter=False, showCrosshair=True)


In [ ]:
roi

In [8]:
x, y, width, height = roi
track_window = (x, y, width, height) 

In [9]:
 
# set up the Region of 
# Interest for tracking 
roi = frame[y:y + height, x : x + width] 

In [ ]:
roi.shape

In [11]:
import matplotlib.pyplot as plt
def plot_array(fillter):
    plt.imshow(cv.cvtColor(fillter, cv.COLOR_BGR2RGB))
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_array(roi)

In [ ]:
# convert ROI from BGR to 
# HSV format 
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV) 
plot_array(hsv_roi)  

In [ ]:
# Click on a pixel to get its HSV values
def pick_color(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        pixel = hsv_roi[y, x]  # Get HSV values at clicked point
        print(f"HSV at ({x}, {y}): {pixel}")
        
cv.imshow("Select a Pixel", frame)
cv.setMouseCallback("Select a Pixel", pick_color)

cv.waitKey(0)

In [ ]:
# perform masking operation 
mask = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255))) 
plot_array(mask)  

In [ ]:



roi_hist = cv.calcHist([hsv_roi],  
                       [0], mask, 
                       [180],  
                       [0, 180]) 
  
cv.normalize(roi_hist, roi_hist, 
             0, 255, cv.NORM_MINMAX) 

In [20]:
term_crit = ( cv.TERM_CRITERIA_EPS |  
             cv.TERM_CRITERIA_COUNT, 15, 2) 
  

In [ ]:

while(1): 
      
    ret, frame = cap.read() 
      
    # Resize the video frames. 
    # frame = cv.resize(frame,  (720, 720), fx = 0, fy = 0, interpolation = cv.INTER_CUBIC) 
      
    cv.imshow('Original', frame) 
  
    # perform thresholding on  
    # the video frames 
    ret1, frame1 = cv.threshold(frame, 
                                180, 155, 
                                cv.THRESH_TOZERO_INV) 
  
    # convert from BGR to HSV 
    # format. 
    hsv = cv.cvtColor(frame1,  
                      cv.COLOR_BGR2HSV) 
  
    dst = cv.calcBackProject([hsv],  
                             [0],  
                             roi_hist,  
                             [0, 180], 1) 
      
    # apply Camshift to get the  
    # new location 
    ret2, track_window = cv.CamShift(dst, 
                                     track_window, 
                                     term_crit) 
  
    # Draw it on image 
    pts = cv.boxPoints(ret2) 
      
    # convert from floating 
    # to integer 
    pts = np.int0(pts) 
  
    # Draw Tracking window on the 
    # video frame. 
    Result = cv.polylines(frame,  
                          [pts],  
                          True,  
                          (0, 255, 255),  
                          2) 
  
    cv.imshow('Camshift', Result) 
  
    # set ESC key as the 
    # exit button. 
    k = cv.waitKey(30) & 0xff
      
    if k == 27: 
        break
          
  
# Release the cap object 
cap.release() 
  
# close all opened windows 
cv.destroyAllWindows() 

## Challenges